In [8]:
from google.colab import files

In [9]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sfmpds","key":"d3e3666625dd8e008b8c83c7eaddc345"}'}

In [10]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle datasets download -d mishra5001/credit-card

 85% 96.0M/112M [00:00<00:00, 217MB/s]
100% 112M/112M [00:00<00:00, 148MB/s] 


In [13]:
! unzip credit-card.zip

Archive:  credit-card.zip
  inflating: application_data.csv    
  inflating: columns_description.csv  
  inflating: previous_application.csv  


In [ ]:
! pip install optbinning

In [ ]:
! pip install ray

In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ray
import psutil
import tqdm


from optbinning import OptimalBinning
from sklearn.preprocessing  import LabelEncoder
import time

plt.style.use("ggplot")

In [5]:
psutil.cpu_count()

2

In [14]:
df_application = pd.read_csv('./application_data.csv')
df_application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# select only cash loans
df_cash = df_application[df_application['NAME_CONTRACT_TYPE'] == 'Cash loans']
df_cash.shape

(278232, 122)

In [16]:
df_cash.columns = df_cash.columns.str.lower()
df_cash.columns

Index(['sk_id_curr', 'target', 'name_contract_type', 'code_gender',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'amt_credit', 'amt_annuity',
       ...
       'flag_document_18', 'flag_document_19', 'flag_document_20',
       'flag_document_21', 'amt_req_credit_bureau_hour',
       'amt_req_credit_bureau_day', 'amt_req_credit_bureau_week',
       'amt_req_credit_bureau_mon', 'amt_req_credit_bureau_qrt',
       'amt_req_credit_bureau_year'],
      dtype='object', length=122)

In [17]:
# we have many attributes and my computer ... well, let's just say he's had his best days lol
# so, let's apply feature selectin methods

# verify if we have duplicates by id
df_cash.groupby('sk_id_curr')['target'].size().sort_values(ascending = False)

sk_id_curr
100002    1
337513    1
337495    1
337496    1
337497    1
         ..
218945    1
218946    1
218947    1
218950    1
456255    1
Name: target, Length: 278232, dtype: int64

In [18]:
attr_columns = [x for x in df_cash.columns if x not in ['sk_id_curr', 'target', 'name_contract_type', 'code_gender']]
len(attr_columns)

118

In [61]:
@ray.remote
def higher_freq_attr(df, attr_to_eval, freq_thr = 0.95, numb_cpus:int=2):
  '''
    Function to evaluate the frequency of attributes. Attributes with 
    frequencies higher than 'freq_thr' are returned for later removal.

    Parameters:
      df:Dataframe.
        Pandas dataframe with attributes.

      attr_to_eval:list.
        List of attributes that will be evaluated.
        
      freq_thr:float
        Maximum value for the frequencies of values of an attribute. Attributes 
        with value frequencies above the threshold will be removed.
    
    Returns:
      attr_to_drop:list
        List of the attributes that will be removed.
  '''
  ray.init(num_cpus = numb_cpus)
  attr_to_drop = []
  #tic = time()
  for attr in attr_to_eval:
    class_freq = list(np.around(df.loc[:, attr].value_counts(normalize = True), 2))

    for freq in class_freq:
      if freq >= freq_thr:
        attr_to_drop.append(attr)
        break;

  #tac = time()
  ray.shutdown()
  return(attr_to_drop)

In [ ]:
tic = time.time()
attr_to_drop = higher_freq_attr.remote(df_cash, attr_columns)
tac = time.time()

In [26]:
# time using ray 
tac - tic

0.3419678211212158

In [30]:
# time without ray
tac - tic

1.3707938194274902

In [31]:
# the suffixe 'f' represent the number of filter applied
df_f1 = df_cash.drop(columns = attr_to_drop)
df_f1.shape

(278232, 95)

In [32]:
df_f1.head()

,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,...,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_3,flag_document_6,flag_document_8,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,2.0,2.0,2.0,-1134.0,1,0,0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,1.0,0.0,-828.0,1,0,0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,2.0,0.0,-617.0,1,0,0,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,-1106.0,0,0,1,0.0,0.0,0.0
5,100008,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,0.0,0.0,0.0,-2536.0,1,0,0,0.0,1.0,1.0


In [33]:
df_f1.select_dtypes(include = ['int64', 'float64']).head()

,sk_id_curr,target,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,region_population_relative,days_birth,days_employed,...,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_3,flag_document_6,flag_document_8,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,2.0,2.0,2.0,-1134.0,1,0,0,0.0,0.0,1.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0.0,1.0,0.0,-828.0,1,0,0,0.0,0.0,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0.0,2.0,0.0,-617.0,1,0,0,NaN,NaN,NaN
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0.0,0.0,0.0,-1106.0,0,0,1,0.0,0.0,0.0
5,100008,0,0,99000.0,490495.5,27517.5,454500.0,0.035792,-16941,-1588,...,0.0,0.0,0.0,-2536.0,1,0,0,0.0,1.0,1.0


In [34]:
# transform flag attr to boolean
attr_to_bool = df_f1.columns[df_f1.columns.str.startswith('flag_')]
attr_to_bool

Index(['flag_own_car', 'flag_own_realty', 'flag_emp_phone', 'flag_work_phone',
       'flag_phone', 'flag_email', 'flag_document_3', 'flag_document_6',
       'flag_document_8'],
      dtype='object')

In [35]:
for attr in attr_to_bool:
    df_f1[attr] = df_f1[attr].astype('bool')

In [36]:
df_f1.select_dtypes(include = ['bool']).head()

,flag_own_car,flag_own_realty,flag_emp_phone,flag_work_phone,flag_phone,flag_email,flag_document_3,flag_document_6,flag_document_8
0,True,True,True,False,True,False,True,False,False
1,True,True,True,False,True,False,True,False,False
3,True,True,True,False,False,False,True,False,False
4,True,True,True,False,False,False,False,False,True
5,True,True,True,True,True,False,True,False,False


In [37]:
df_f1.select_dtypes(include = ['O']).head()

,name_contract_type,code_gender,name_type_suite,name_income_type,name_education_type,name_family_status,name_housing_type,occupation_type,weekday_appr_process_start,organization_type,fondkapremont_mode,wallsmaterial_mode
0,Cash loans,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,reg oper account,"Stone, brick"
1,Cash loans,F,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School,reg oper account,Block
3,Cash loans,F,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3,NaN,NaN
4,Cash loans,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion,NaN,NaN
5,Cash loans,M,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,Laborers,WEDNESDAY,Other,NaN,NaN


In [63]:
type(df_f1)

pandas.core.frame.DataFrame

In [64]:
@ray.remote
def iv_filter(df:pd.core.frame.DataFrame, attr_to_eval:list, target:str='target', iv_thr:float = 0.02, dtype:str='numerical', numb_cpus:int=2):
  '''
    Function to filter attributes according to their IV (Information Value). This 
    function creates bins for non-categorical attributes or recreates bins for 
    categorical attributes and calculates their respective IV. It returns a list 
    of attributes that have IV above the defined minimum (iv_thr). The optbinning 
    library is used to create the bins. To learn more about binning process:
    http://gnpalencia.org/optbinning/index.html. To learn more about ray:
    https://docs.ray.io/en/latest/index.html
    
    Parameters:
      df: DataFrame.
        Dataframe with attributes to create bins

      attr_to_evaluate: list.
        List of attributes to create bins
      
      target: str.
        Name of target variable in the DataFrame.
      
      iv_thr: int (default = 0.02)
        Minimum IV value.
      
      dtype: str.
        OptimalBinning parameter. Determines the data type of the attributes 
        that the bins will be created.

      Returns:
        attr_info: dict.
          Dictionary containing a list of the attributes that do not have IV 
          above iv_thr and a dictionary of the categories created for each 
          attribute that have IV above iv_thr.
  '''
  ray.init(num_cpus = numb_cpus, ignore_reinit_error=True)
  attr_to_drop = []
  attr_class = {}

  for attr in attr_to_eval:
    #print("Evaluating:", attr)
    x = df[attr].values
    y = df[target]
    
    optb = OptimalBinning(name=attr, dtype=dtype, solver="mip", cat_cutoff=0.05)
    optb.fit(x, y)

    binning_table = optb.binning_table.build(4)
    binning_table.loc[:, 'Bin'] = binning_table.Bin.astype('str')
    
    iv = float(binning_table.iloc[-1:,7])
    #print("IV:",iv)
    
    if iv < iv_thr:
      attr_to_drop.append(attr)
    else:
      bins_splits = list(optb.splits)
      bins_splits.insert(0, -np.inf)
      bins_splits.append(np.inf)
      bins = list(binning_table.Bin.unique())[:-3]

      bins_config = {"bins_split":bins_splits,
                     "bins":bins
                     }
      attr_class[attr] = bins_config

  ray.shutdown()

  attr_info = {'attr_to_drop': attr_to_drop,
               'attr_class': attr_class}

  return(attr_info)

In [65]:
attr_num = list(df_f1.select_dtypes(include = ['int64', 'float64']).columns)
attr_num = [x for x in attr_num if x not in ['sk_id_curr', 'target']]

In [66]:
tic = time.time()
attr_info = iv_filter.remote(df=df_f1, attr_to_eval=attr_num)
tac = time.time()

In [42]:
# time without ray
tac - tic

9.92355728149414

In [67]:
# time using ray
tac - tic

0.32030510902404785

In [70]:
attr_to_drop = ray.get(attr_info)['attr_to_drop']
attr_class_num = ray.get(attr_info)['attr_class']

In [69]:
attr_to_drop

['cnt_children',
 'amt_income_total',
 'cnt_fam_members',
 'hour_appr_process_start',
 'live_city_not_work_city',
 'years_build_avg',
 'commonarea_avg',
 'floorsmin_avg',
 'landarea_avg',
 'livingapartments_avg',
 'nonlivingapartments_avg',
 'basementarea_mode',
 'years_build_mode',
 'commonarea_mode',
 'floorsmin_mode',
 'landarea_mode',
 'livingapartments_mode',
 'nonlivingapartments_mode',
 'years_build_medi',
 'commonarea_medi',
 'floorsmin_medi',
 'landarea_medi',
 'livingapartments_medi',
 'nonlivingapartments_medi',
 'obs_30_cnt_social_circle',
 'def_30_cnt_social_circle',
 'obs_60_cnt_social_circle',
 'def_60_cnt_social_circle',
 'amt_req_credit_bureau_mon',
 'amt_req_credit_bureau_qrt',
 'amt_req_credit_bureau_year']

In [ ]:
attr_class_num

In [72]:
# drop not iv significant numerical attributes
df_f2 = df_f1.drop(columns = attr_to_drop)
df_f2.shape

(278232, 64)

In [73]:
df_f2.head()

,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,amt_credit,amt_annuity,amt_goods_price,name_type_suite,...,floorsmax_medi,livingarea_medi,nonlivingarea_medi,fondkapremont_mode,totalarea_mode,wallsmaterial_mode,days_last_phone_change,flag_document_3,flag_document_6,flag_document_8
0,100002,1,Cash loans,M,True,True,406597.5,24700.5,351000.0,Unaccompanied,...,0.0833,0.0193,0.00,reg oper account,0.0149,"Stone, brick",-1134.0,True,False,False
1,100003,0,Cash loans,F,True,True,1293502.5,35698.5,1129500.0,Family,...,0.2917,0.0558,0.01,reg oper account,0.0714,Block,-828.0,True,False,False
3,100006,0,Cash loans,F,True,True,312682.5,29686.5,297000.0,Unaccompanied,...,NaN,NaN,NaN,NaN,NaN,NaN,-617.0,True,False,False
4,100007,0,Cash loans,M,True,True,513000.0,21865.5,513000.0,Unaccompanied,...,NaN,NaN,NaN,NaN,NaN,NaN,-1106.0,False,False,True
5,100008,0,Cash loans,M,True,True,490495.5,27517.5,454500.0,"Spouse, partner",...,NaN,NaN,NaN,NaN,NaN,NaN,-2536.0,True,False,False


In [75]:
attr_cat = list(df_f1.select_dtypes(include = ['O', 'bool']).columns)
attr_info_cat = iv_filter.remote(df=df_f2, attr_to_eval=attr_cat, dtype='categorical')

(iv_filter pid=551) Traceback (most recent call last):
(iv_filter pid=551)   File "python/ray/_raylet.pyx", line 804, in ray._raylet.task_execution_handler
(iv_filter pid=551)   File "python/ray/_raylet.pyx", line 510, in ray._raylet.execute_task
(iv_filter pid=551) AttributeError: 'Worker' object has no attribute 'core_worker'
(iv_filter pid=551) 
(iv_filter pid=551) During handling of the above exception, another exception occurred:
(iv_filter pid=551) 
(iv_filter pid=551) Traceback (most recent call last):
(iv_filter pid=551)   File "python/ray/_raylet.pyx", line 831, in ray._raylet.task_execution_handler
(iv_filter pid=551)   File "/usr/local/lib/python3.7/dist-packages/ray/_private/utils.py", line 172, in push_error_to_driver
(iv_filter pid=551)     worker.core_worker.push_error(job_id, error_type, message, time.time())
(iv_filter pid=551) AttributeError: 'Worker' object has no attribute 'core_worker'
(iv_filter pid=551) Exception ignored in: 'ray._raylet.task_execution_handler'
(

In [79]:
attr_to_drop_cat = ray.get(attr_info_cat)['attr_to_drop']
attr_class__cat = ray.get(attr_info_cat)['attr_class']

In [80]:
len(attr_to_drop_cat)

13

In [81]:
# drop not iv significant categorical attributes
df_f3 = df_f2.drop(columns = attr_to_drop_cat)
df_f3.shape

(278232, 51)

In [82]:
df_f3.head()

,sk_id_curr,target,code_gender,amt_credit,amt_annuity,amt_goods_price,name_income_type,name_education_type,name_family_status,region_population_relative,...,basementarea_medi,years_beginexpluatation_medi,elevators_medi,entrances_medi,floorsmax_medi,livingarea_medi,nonlivingarea_medi,totalarea_mode,wallsmaterial_mode,days_last_phone_change
0,100002,1,M,406597.5,24700.5,351000.0,Working,Secondary / secondary special,Single / not married,0.018801,...,0.0369,0.9722,0.00,0.0690,0.0833,0.0193,0.00,0.0149,"Stone, brick",-1134.0
1,100003,0,F,1293502.5,35698.5,1129500.0,State servant,Higher education,Married,0.003541,...,0.0529,0.9851,0.08,0.0345,0.2917,0.0558,0.01,0.0714,Block,-828.0
3,100006,0,F,312682.5,29686.5,297000.0,Working,Secondary / secondary special,Civil marriage,0.008019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-617.0
4,100007,0,M,513000.0,21865.5,513000.0,Working,Secondary / secondary special,Single / not married,0.028663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1106.0
5,100008,0,M,490495.5,27517.5,454500.0,State servant,Secondary / secondary special,Married,0.035792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2536.0


In [83]:
attr_to_model = list(df_f3.columns)
attr_to_model = [x for x in attr_to_model if x not in ['sk_id_curr', 'target', 'code_gender']]

In [84]:
df_f3.amt_credit.max()

4050000.0

In [85]:
list(attr_class_num['amt_credit']['bins_split'])

[-inf,
 153540.0,
 284395.5,
 331733.25,
 400446.0,
 452475.0,
 672320.25,
 835555.5,
 1123562.25,
 1345173.75,
 inf]

In [304]:
#attr_class_num.update(attr_class_cat)
#attr_class = attr_class_num


In [305]:
attr_class['region_population_relative']['bins']

['(-inf, 0.0131)',
 '[0.0131, 0.0173)',
 '[0.0173, 0.0194)',
 '[0.0194, 0.0217)',
 '[0.0217, 0.0319)',
 '[0.0319, 0.0410)',
 '[0.0410, inf)']

In [306]:
pd.cut(df_f3[attr], attr_class['region_population_relative']['bins_split'],
      labels=attr_class['region_population_relative']['bins'])

0         [0.0173, 0.0194)
1           (-inf, 0.0131)
3           (-inf, 0.0131)
4         [0.0217, 0.0319)
5         [0.0319, 0.0410)
                ...       
307506    [0.0319, 0.0410)
307507    [0.0217, 0.0319)
307508      (-inf, 0.0131)
307509      (-inf, 0.0131)
307510       [0.0410, inf)
Name: region_population_relative, Length: 278232, dtype: category
Categories (7, object): ['(-inf, 0.0131)' < '[0.0131, 0.0173)' < '[0.0173, 0.0194)' <
                         '[0.0194, 0.0217)' < '[0.0217, 0.0319)' < '[0.0319, 0.0410)' < '[0.0410, inf)']

In [312]:
attr_class['name_income_type']

{'bins_split': [-inf,
  array(['Pensioner'], dtype=object),
  array(['State servant'], dtype=object),
  array(['Commercial associate'], dtype=object),
  array(['Working'], dtype=object),
  array(['Student', 'Unemployed', 'Maternity leave'], dtype=object),
  inf],
 'bins': ["['Pensioner']",
  "['State servant']",
  "['Commercial associate']",
  "['Working']",
  "['Student' 'Unemployed' 'Maternity leave']"]}

In [311]:
df_cat = df_f3.copy()

for attr in attr_class.keys():
  print(attr)
  if attr != 'code_gender':
    df_cat[attr] = pd.cut(df_f3[attr], attr_class[attr]['bins_split'],
        labels=attr_class[attr]['bins'])

df_cat.head()

amt_credit
amt_annuity
amt_goods_price
region_population_relative
days_birth
days_employed
days_registration
days_id_publish
own_car_age
region_rating_client
region_rating_client_w_city
reg_city_not_live_city
reg_city_not_work_city
ext_source_1
ext_source_2
ext_source_3
apartments_avg
basementarea_avg
years_beginexpluatation_avg
elevators_avg
entrances_avg
floorsmax_avg
livingarea_avg
nonlivingarea_avg
apartments_mode
years_beginexpluatation_mode
elevators_mode
entrances_mode
floorsmax_mode
livingarea_mode
nonlivingarea_mode
apartments_medi
basementarea_medi
years_beginexpluatation_medi
elevators_medi
entrances_medi
floorsmax_medi
livingarea_medi
nonlivingarea_medi
totalarea_mode
days_last_phone_change
code_gender
name_income_type


/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/tile.py:280: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bins = np.asarray(bins)


ValueError: ignored

In [151]:
attr_class_num['amt_credit']['bins'][:-4]

['(-inf, 153540.00)',
 '[153540.00, 284395.50)',
 '[284395.50, 331733.25)',
 '[331733.25, 400446.00)',
 '[400446.00, 452475.00)',
 '[452475.00, 672320.25)',
 '[672320.25, 835555.50)',
 '[835555.50, 1123562.25)',
 '[1123562.25, 1345173.75)']

In [188]:
len(a)

10

In [232]:
len(attr_class_num['amt_credit']['bins'])

10